In [1]:
# Importing packages

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from tqdm import tqdm


The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [2]:
def checktime():
    print(datetime.datetime.now()) 
data_path = "/ocean/projects/atm200007p/jlin96/longSPrun/"

In [3]:
ds = ! ls {data_path} | grep 'h1.0000-0[12]-01-00000.nc'
ds = [data_path + i for i in ds]

test_data = xr.open_mfdataset(ds)
# from collections import OrderedDict
# list(OrderedDict.fromkeys(test_data.data_vars.variables))

In [4]:
nntbp = test_data['NNTBP'].values
nnqbp = test_data["NNQBP"].values
ps = test_data["NNPS"].values

p0 = test_data["P0"].values
hyam = test_data["hyam"].values
hybm = test_data["hybm"].values
relhum = test_data["RELHUM"].values

In [5]:
print('nntbp.shape:',nntbp.shape) # 4-D variables (timesteps, level, latitude, longitide)
print('nnqbp.shape:',nnqbp.shape) # 4-D variables (timesteps, level, latitude, longitide)
print('ps.shape:',ps.shape) # 3-D variables (timesteps, latitude, longitide)
print('p0.shape:',p0.shape) # 0
print('hyam.shape:',hyam.shape) # (level, ) 
print('hybm.shape:',hybm.shape) # (level, )
print('relhum.shape:',relhum.shape) # 4-D variables (timesteps, level, latitude, longitide)

nntbp.shape: (96, 30, 64, 128)
nnqbp.shape: (96, 30, 64, 128)
ps.shape: (96, 64, 128)
p0.shape: (96,)
hyam.shape: (96, 30)
hybm.shape: (96, 30)
relhum.shape: (96, 30, 64, 128)


# TODO
* Task:
Find a efficient way to do preprocessing, rewrite the calculation process of relative humidity.
    
* Question:
How to multiply a four dimensional python numpy array by a vector indexed by second dimension?


In [7]:
# Relative Humidity Conversion Functions

def eliq(T):
    a_liq = np.array([-0.976195544e-15,-0.952447341e-13,\
                                 0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]);
    c_liq = -80.0
    T0 = 273.16
    return 100.0*np.polyval(a_liq,np.maximum(c_liq,T-T0))

def eice(T):
    a_ice = np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]);
    c_ice = np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07])
    T0 = 273.16
    return np.where(T>c_ice[0],eliq(T),\
                   np.where(T<=c_ice[1],100.0*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])),100.0*np.polyval(a_ice,T-T0)))

def esat(T):
    T0 = 273.16
    T00 = 253.16
    omtmp = (T-T00)/(T0-T00)
    omega = np.maximum(0.0,np.minimum(1.0,omtmp))
    return np.where(T>T0,eliq(T),np.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))

def RH(T,qv,P0,PS,hyam,hybm):
    R = 287.0
    Rv = 461.0
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    return Rv*p*qv/(R*esat(T))




# Decomposition the calculation of relative humidty 

## The usage of np.functions

1. np.where(condition, x,y)

Return x when condition is true, otherwise return y

2. np.polyvar(p, b)
p can be list with n elements, return $p[0]*b**(N-1) + p[1]*b**(N-2) + ... + p[N-2]*b + p[N-1]$

3. np.maximum(a,b)
return the larger element of a and b, this function can be broadcast to (n-d)array
example, a = [1,2,5], b=[3,1,2], return [3,2,5]


## The function of eice(T)
```
# Decomposition 
# The nest np.where function
y1 = 100.0*np.polyval(a_ice,T-T0)
x1 = 100.0*(c_ice[3]+np.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+np.maximum(c_ice[2],T-T0)*c_ice[5])
con1 = T<=c_ice[1] # condition 1
y0 = np.where(con1, x1, y1) # return x1 when cond1 is true, otherwise return y1, the return value assign to y0

# The first np.where function
eliq(T) = 100.0*np.polyval(a_liq,np.maximum(c_liq,T-T0)) # Calculate the polyval of the maximun element between c_lip and (T-T0)
x0 = eliq(T)
con0 = T>c_ice[0]
np.where(con0, x0, y0) #return x0 if con0 is true, otherwise return y0
```
## The function of esat(T)

```
# Decomposition
# The nest np.where function
y2 = omega*eliq(T)+(1-omega)*eice(T)
x2 = eice(T)
con2 = T<T00
y3 = np.where(con2, x2, y2) #return x2 if con2 is true, otherwise return y2
# The first np.where function
x3 = eliq(T)
con3 = T>T0
np.where(con3, x3, y3) 
```

## The function of RH



```
def RH(T,qv,P0,PS,hyam,hybm):
    R = 287.0
    Rv = 461.0
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    return Rv*p*qv/(R*esat(T))
```
Analysis the dimension of variables

* ps.shape: (96, 64, 128)
* p0.shape: (96,)
* hyam.shape: (96, 30)
* hybm.shape: (96, 30)
* relhum.shape: (96, 30, 64, 128) # (time, level, latitude, logitude)

Since hyam and hybm are 2-D variables (time, level), mean that given the same time and level, values are constant in every latitude and longitude. The original verison used to convert all variables into 2-D arrays using nest for loop.
How to calculate the new_humidity without for loop?

More specific, How to calculate all variables only for (time, level) if the variables remain 4-D arrays?



## Comparsion

The 2-D with for loop and 4-D without for loop

* results
* time consuming

In [30]:
# 2-D with nest for loop
spData = test_data.copy()

%time
nntbp = spData["NNTBP"].values
nnqbp = spData["NNQBP"].values
ps = spData["NNPS"].values

p0 = spData["P0"].values
hyam = spData["hyam"].values
hybm = spData["hybm"].values
relhum = spData["RELHUM"].values

p0 = np.array(list(set(p0)))
print("loaded in data")
newhum = np.zeros((spData["time"].shape[0],\
                              spData["lev"].shape[0], \
                              spData["lat"].shape[0], \
                              spData["lon"].shape[0]))
lats = spData["lat"]
lons = spData["lon"]
print("starting for loop")
for i in tqdm(range(len(lats))): # latitude loop
    for j in range(len(lons)): # longotude loop
        latIndex = i
        lonIndex = j
        R = 287.0
        Rv = 461.0
        # p0 is a scale, hyam's and hybm's dim are (time, level), ps's dim is (time, latitude, longitude)
        p = p0 * hyam + ps[:, None, latIndex, lonIndex] * hybm # Total pressure (Pa) #(96,30)
        T = nntbp[:, :, latIndex, lonIndex] #(96,30)
        qv = nnqbp[:, :, latIndex, lonIndex] #(96,30)
        newhum[:,:, latIndex, lonIndex] = Rv*p*qv/(R*esat(T)) #(96,30)


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
loaded in data
starting for loop


100%|██████████| 64/64 [00:25<00:00,  2.54it/s]


In [35]:
print(newhum.shape)
newhum[1]

(96, 30, 64, 128)


array([[[2.07679620e-05, 2.07765116e-05, 2.07846297e-05, ...,
         2.07387211e-05, 2.07490092e-05, 2.07587584e-05],
        [2.07244073e-05, 2.07233199e-05, 2.07264949e-05, ...,
         2.07493677e-05, 2.07383702e-05, 2.07297042e-05],
        [2.07122006e-05, 2.07276383e-05, 2.07635703e-05, ...,
         2.07809747e-05, 2.07430959e-05, 2.07182621e-05],
        ...,
        [8.57701712e-01, 8.57919811e-01, 8.58099526e-01, ...,
         8.56884253e-01, 8.57177307e-01, 8.57451498e-01],
        [8.58699381e-01, 8.58704660e-01, 8.58708510e-01, ...,
         8.58676614e-01, 8.58685413e-01, 8.58693002e-01],
        [8.59636173e-01, 8.59642882e-01, 8.59652066e-01, ...,
         8.59626880e-01, 8.59628419e-01, 8.59631554e-01]],

       [[2.07307954e-04, 2.07192194e-04, 2.07072140e-04, ...,
         2.07665110e-04, 2.07539941e-04, 2.07421211e-04],
        [2.03294615e-04, 2.03541076e-04, 2.03802193e-04, ...,
         2.02714745e-04, 2.02857483e-04, 2.03055085e-04],
        [1.98225844e-04, 

In [43]:
%time
newhum_1 = np.zeros((spData["time"].shape[0],\
                  spData["lev"].shape[0], \
                  spData["lat"].shape[0], \
                  spData["lon"].shape[0]))
# 4-D without for loop
R = 287.0
Rv = 461.0

p = p0 * hyam[:,:,np.newaxis, np.newaxis] + ps[:, np.newaxis, :, :] * hybm[:,:,np.newaxis, np.newaxis] # Total pressure (Pa) #(96,30
newhum_1 = Rv*p*nnqbp/(R*esat(nntbp))
newhum_1

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 4.29 µs
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


array([[[[2.07431854e-05, 2.07503544e-05, 2.07570936e-05, ...,
          2.07179005e-05, 2.07269103e-05, 2.07353002e-05],
         [2.07001925e-05, 2.06996245e-05, 2.07024919e-05, ...,
          2.07197841e-05, 2.07110203e-05, 2.07041518e-05],
         [2.06523295e-05, 2.06673045e-05, 2.07018693e-05, ...,
          2.07173134e-05, 2.06813710e-05, 2.06578409e-05],
         ...,
         [8.57780076e-01, 8.57972054e-01, 8.58127077e-01, ...,
          8.57033557e-01, 8.57303404e-01, 8.57554773e-01],
         [8.58648183e-01, 8.58651813e-01, 8.58654012e-01, ...,
          8.58630861e-01, 8.58637240e-01, 8.58643179e-01],
         [8.59605598e-01, 8.59613242e-01, 8.59623085e-01, ...,
          8.59596359e-01, 8.59597239e-01, 8.59600098e-01]],

        [[2.07155626e-04, 2.07034740e-04, 2.06906953e-04, ...,
          2.07547733e-04, 2.07418309e-04, 2.07289984e-04],
         [2.02832201e-04, 2.03069046e-04, 2.03316980e-04, ...,
          2.02348239e-04, 2.02458067e-04, 2.02625161e-04],
        

In [39]:
newhum_1 - newhum # The results are the same, so we can replace for loop to save time

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

## Test on larger datasets
Let's use the two months datasets to test the time consuming

In [47]:
ds = ! ls {data_path} | grep 'h1.0000-0[12]'
ds = [data_path + i for i in ds]
ds

['/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.0000-01-01-00000.nc',
 '/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.0000-01-02-00000.nc',
 '/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.0000-01-03-00000.nc',
 '/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.0000-01-04-00000.nc',
 '/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.0000-01-05-00000.nc',
 '/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.0000-01-06-00000.nc',
 '/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.0000-01-07-00000.nc',
 '/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.0000-01-08-00000.nc',
 '/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.0000-01-09-00000.nc',
 '/ocean/projects/atm200007p/jlin96/longSPrun/AndKua_aqua_Base_training.cam2.h1.00

In [51]:
%% time
two_month_data = xr.open_mfdataset(ds)
two_month_data

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.58 µs


<xarray.Dataset>
Dimensions:       (time: 2832, lat: 64, lon: 128, crm_x: 32, crm_y: 1,
                   crm_z: 28, lev: 30, ilev: 31, isccp_prs: 7, isccp_tau: 7,
                   isccp_prstau: 49, tbnd: 2)
Coordinates:
  * lat           (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon           (lon) float64 0.0 2.812 5.625 8.438 ... 351.6 354.4 357.2
  * crm_x         (crm_x) float64 0.0 4.0 8.0 12.0 ... 112.0 116.0 120.0 124.0
  * crm_y         (crm_y) float64 0.0
  * crm_z         (crm_z) float64 992.6 976.3 957.5 936.2 ... 38.27 24.61 14.36
  * lev           (lev) float64 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * ilev          (ilev) float64 2.255 5.032 10.16 18.56 ... 967.5 985.1 1e+03
  * isccp_prs     (isccp_prs) float64 90.0 245.0 375.0 500.0 620.0 740.0 900.0
  * isccp_tau     (isccp_tau) float64 0.15 0.8 2.45 6.5 16.2 41.5 219.5
  * isccp_prstau  (isccp_prstau) float64 90.0 90.0 90.0 ... 900.0 900.0 900.2
  * time          (time) object 0000-01-01 00:00:00 ... 0000-02-28 23:30:00
Dimensions without coordinates: tbnd
Data variables: (12/41)
    P0            (time) float64 1e+05 1e+05 1e+05 1e+05 ... 1e+05 1e+05 1e+05
    time_bnds     (time, tbnd) object dask.array<chunksize=(48, 2), meta=np.ndarray>
    date_written  (time) |S8 dask.array<chunksize=(48,), meta=np.ndarray>
    time_written  (time) |S8 dask.array<chunksize=(48,), meta=np.ndarray>
    ntrm          (time) int32 63 63 63 63 63 63 63 63 ... 63 63 63 63 63 63 63
    ntrn          (time) int32 63 63 63 63 63 63 63 63 ... 63 63 63 63 63 63 63
    ...            ...
    RELHUM        (time, lev, lat, lon) float32 dask.array<chunksize=(48, 30, 64, 128), meta=np.ndarray>
    SHFLX         (time, lat, lon) float32 dask.array<chunksize=(48, 64, 128), meta=np.ndarray>
    SOLIN         (time, lat, lon) float32 dask.array<chunksize=(48, 64, 128), meta=np.ndarray>
    SPDQ          (time, lev, lat, lon) float32 dask.array<chunksize=(48, 30, 64, 128), meta=np.ndarray>
    SPDT          (time, lev, lat, lon) float32 dask.array<chunksize=(48, 30, 64, 128), meta=np.ndarray>
    TPHYSTND      (time, lev, lat, lon) float32 dask.array<chunksize=(48, 30, 64, 128), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    source:       CAM
    case:         AndKua_aqua_Base_training
    title:        
    logname:      jlin96
    host:         
    Version:      $Name:  $
    revision_Id:  $Id: history.F90,v 1.26.2.38 2003/12/15 18:52:35 hender Exp $

## Testing Result
It takes too long time for loading the datasets. Maybe we can find some more efficient way to process it.
Fortunately, the calculation for new_humidity just takes **71s for two month datasets**, so faster compare to previous.

In [63]:
spData = two_month_data.copy()

nntbp = spData["NNTBP"].values
nnqbp = spData["NNQBP"].values
ps = spData["NNPS"].values

p0 = spData["P0"].values
hyam = spData["hyam"].values
hybm = spData["hybm"].values
relhum = spData["RELHUM"].values

newhum_2 = np.zeros((spData["time"].shape[0],\
                  spData["lev"].shape[0], \
                  spData["lat"].shape[0], \
                  spData["lon"].shape[0]))

# 4-D without for loop
R = 287.0
Rv = 461.0
p0 = np.array(list(set(p0)))
p = p0 * hyam[:,:,np.newaxis, np.newaxis] + ps[:, np.newaxis, :, :] * hybm[:,:,np.newaxis, np.newaxis] # Total pressure (Pa) #(96,30
%time newhum_2 = Rv*p*nnqbp/(R*esat(nntbp))


2022-11-20 13:25:21.461643


datetime.timedelta(seconds=71, microseconds=93315)

## Load data in more efficient way
* load data by chunk

In [4]:
# Load two month dataset
import datetime
import xarray as xr
data_path = "/ocean/projects/atm200007p/jlin96/longSPrun/" # load one year dataset
ds = ! ls {data_path} | grep 'h1.0000-*'
ds = [data_path + i for i in ds]
%time one_year_data = xr.open_mfdataset(ds, decode_times=True, chunks={'time': 960, 'latitude':1, 'longitude':1}, parallel=True)


KeyboardInterrupt: 

For one year dataset, the following code for loading data costs 155s.
```
xr.open_mfdataset(ds, decode_times=True, chunks={'time': 1000, 'latitude':1, 'longitude':1})
```
But when we add `parallel=True` in the end, it just cost 76s to load all data. 

## Rewrite make_nn_input function and Load_data


In [8]:
def load_data(month, year, data_path):
    datasets = ls(data_path)
    month = str(month).zfill(2)
    year = str(year).zfill(4)
    datasets = [data_path + x for x in datasets if "h1." + year + "-" + month in x]
    ds = xr.open_mfdataset(ds, decode_times=True, chunks={'time': 960, 'latitude':1, 'longitude':1}, parallel=True)
    return ds


def make_nn_input(spData, family='relative', subsample = False, spacing = 8, contiguous = True, print_diagnostics = True):
    
    if subsample:
        nnInput = spData[:,:,:,sample_indices(spData.shape[3], spacing, True)]
        
    nntbp = spData["NNTBP"].values
    nnqbp = spData["NNQBP"].values
    ps = spData["NNPS"].values
    
    p0 = spData["P0"].values
    hyam = spData["hyam"].values
    hybm = spData["hybm"].values
    relhum = spData["RELHUM"].values

    p0 = np.array(list(set(p0)))
    print("loaded in data")
    newhum = np.zeros((spData["time"].shape[0],\
                                  spData["lev"].shape[0], \
                                  spData["lat"].shape[0], \
                                  spData["lon"].shape[0]))

    p = p0 * hyam[:,:,np.newaxis, np.newaxis] + \
        ps[:, np.newaxis, :, :] * hybm[:,:,np.newaxis, np.newaxis] # Total pressure (Pa) 
    newhum = Rv*p*nnqbp/(R*esat(nntbp))
    
    nntbp = np.moveaxis(nntbp[1:,:,:,:],0,1)
    nnqbp = np.moveaxis(nnqbp[1:,:,:,:],0,1)
    ps = spData["NNPS"].values[np.newaxis,1:,:,:]
    solin = spData["SOLIN"].values[np.newaxis,1:,:,:] 
    shflx = spData["SHFLX"].values[np.newaxis,:-1,:,:]
    lhflx = spData["LHFLX"].values[np.newaxis,:-1,:,:]
    
    newhum = np.moveaxis(newhum[1:,:,:,:],0,1)    
    oldhum = np.moveaxis(relhum[1:,:,:,:],0,1)

    if family == "specific":
        nnInput = np.concatenate((nntbp, \
                                  nnqbp, \
                                  ps, \
                                  solin, \
                                  shflx, \
                                  lhflx))
    
    elif family == "relative":
        nnInput = np.concatenate((nntbp, \
                                  newhum, \
                                  ps, \
                                  solin, \
                                  shflx, \
                                  lhflx))              
    
    if not contiguous:
        nnInput = nnInput[:,:-1,:,:] #the last timestep of a run can have funky values
        
    if print_diagnostics:
        print("nntbp")
        print(nntbp.shape)
        print("nnqbp")
        print(nnqbp.shape)
        print("lhflx")
        print(lhflx.shape)
        print("shflx")
        print(shflx.shape)
        print("ps")
        print(ps.shape)
        print("solin")
        print(solin.shape)
        print("newhum")
        print(newhum.shape)
        print("oldhum")
        print(oldhum.shape)
        print("nnInput")
        print(nnInput.shape)
        errors = (newhum-oldhum/100).flatten()
        result = "Mean relative humidity conversion error: " + str(np.mean(errors)) + "\n"
        result = result + "Variance for relative humidity conversion error: " + str(np.var(errors)) + "\n"
        result = result + "nntbp.shape: " + str(nntbp.shape) + "\n"
        result = result + "nnqbp.shape: " + str(nnqbp.shape) + "\n"
        result = result + "lhflx.shape: " + str(lhflx.shape) + "\n"
        result = result + "shflx.shape: " + str(shflx.shape) + "\n"
        result = result + "ps.shape: " + str(ps.shape) + "\n"
        result = result + "solin.shape: " + str(solin.shape) + "\n"
        result = result + "newhum.shape: " + str(newhum.shape) + "\n"
        result = result + "oldhum.shape: " + str(oldhum.shape) + "\n"
        result = result + "nnInput.shape: " + str(nnInput.shape) + "\n"
        print(result)
    
    return nnInput

## Test new preprocess for one year dataset without subsampling

It takes so long time to load data. There're two approaches to imporve loading speed but the logic is the same. We divide the data into several chunks, then load them separately. In Jerry's previous version, he implemented a methond to load one-month-data in sequence, and combine all data after 12 months data completed. Here I search for some way to load the data in parallel, which may reduce the time cosuming.

In [3]:
import datetime
import xarray as xr

def checktime():
    print(datetime.datetime.now()) 
data_path = "/ocean/projects/atm200007p/jlin96/longSPrun/"

In [4]:
ds = ! ls {data_path} | grep 'h1.0000-*'
ds = [data_path + i for i in ds]
%time ds = xr.open_mfdataset(ds, decode_times=True, chunks={'time': 960, 'latitude':1, 'longitude':1}, parallel=True)


CPU times: user 12.3 s, sys: 2.06 s, total: 14.4 s
Wall time: 4min 21s


## Test Parallel method 

In [ ]:
def load_data(month):
    path = "/ocean/projects/atm200007p/jlin96/longSPrun/"
    datasets = ls(data_path)
    month = str(month).zfill(2)
    datasets = [data_path + x for x in datasets if "h1.0001"  + "-" + month in x]
    ds = xr.open_mfdataset(ds, decode_times=True, chunks={'time': 960, 'latitude':1, 'longitude':1}, parallel=True)
    return ds

import multiprocessing as mp

pool = mp.Pool(processes=8)
results = pool.map(make_nn_input, range(1,13))

In [ ]:
import time
import os
import xarray as xr

# Read data in parallel 
def f2(day):
    days = "{0:0=2d}".format(day)
    SP_Case = 'SP_newsst_long_fluxout_L125_ERA5_2008_F-MMF1_frontera_ne16pg2_r05_oQU240_CRM1_32_x_120z1200m.5s_crm_nx_rad_16_np_768_nlev_125.frontera'
    SP_PATH = '/media/liran/Data3/SP/'
    print(SP_Case+'.cam.h0.2008-10-'+days+'*.nc')
    print('parent process:', os.getppid())
    print('process id:', os.getpid())
#     SP2008 = xr.open_mfdataset(SP_PATH+SP_Case+'.cam.h0.2008-10-'+days+'*.nc')
    outarray = SP2008.T.values
    print(outarray.shape)
    print('finish id:', os.getpid())
    return outarray

def load_data(data_path):
    
        
if __name__ == '__main__':
    # start 6 worker processes

    with Pool(processes=6) as pool:
        # Should merge those data     
        jj = pool.map(f2, range(1,3))       # I did not use all the pool
        
    
        
    # exiting the 'with'-block has stopped the pool
    print("Now the pool is closed and no longer available")